## 🚀 API Data Retrieval and JSON List Nesting

### Libraries

In [1]:
import pandas as pd
import requests

### Retrieve Data from API using Requests

In [2]:
url = 'https://soda.demo.socrata.com/resource/6yvf-kk3n.json?source=pr&region=Virgin Islands region'

In [3]:
response = requests.get(url).json()

In [4]:
type(response)

list

#### json_normalize (max_level)

In [5]:
response[0:1]

[{'source': 'pr',
  'earthquake_id': '12258002',
  'version': '0',
  'magnitude': '3.1',
  'depth': '25',
  'number_of_stations': '3',
  'region': 'Virgin Islands region',
  'location': {'type': 'Point', 'coordinates': [-64.3056, 19.6145]}}]

In [6]:
df_n1 = pd.json_normalize(response, max_level=1)
df_n1.head()


,source,earthquake_id,version,magnitude,depth,number_of_stations,region,location.type,location.coordinates
0,pr,12258002,0,3.1,25,3,Virgin Islands region,Point,"[-64.3056, 19.6145]"
1,pr,12252002,0,3.1,43,6,Virgin Islands region,Point,"[-64.408, 19.6117]"
2,pr,12256005,0,3.1,47,3,Virgin Islands region,Point,"[-63.8544, 19.3323]"
3,pr,12257002,0,3.1,43,4,Virgin Islands region,Point,"[-64.568, 19.1303]"
4,pr,12252005,0,3.3,60,7,Virgin Islands region,Point,"[-64.17, 19.6184]"


In [7]:
lng = []
for i in range(len(df_n1.iloc[:, 8])):
    lng.append(df_n1.iloc[i, 8][0])

lat = []
for i in range(len(df_n1.iloc[:, 8])):
    lat.append(df_n1.iloc[i, 8][1])


In [36]:
df_n1["longitude"] = lng
df_n1["latitude"] = lat
print(len(df_n1))
df_n1.head()

38


,source,earthquake_id,version,magnitude,depth,number_of_stations,region,location.type,location.coordinates,longitude,latitude
0,pr,12258002,0,3.1,25,3,Virgin Islands region,Point,"[-64.3056, 19.6145]",-64.3056,19.6145
1,pr,12252002,0,3.1,43,6,Virgin Islands region,Point,"[-64.408, 19.6117]",-64.4080,19.6117
2,pr,12256005,0,3.1,47,3,Virgin Islands region,Point,"[-63.8544, 19.3323]",-63.8544,19.3323
3,pr,12257002,0,3.1,43,4,Virgin Islands region,Point,"[-64.568, 19.1303]",-64.5680,19.1303
4,pr,12252005,0,3.3,60,7,Virgin Islands region,Point,"[-64.17, 19.6184]",-64.1700,19.6184


#### json_normalize (record_path + manipulation)

In [14]:
response[0:1]

[{'source': 'pr',
  'earthquake_id': '12258002',
  'version': '0',
  'magnitude': '3.1',
  'depth': '25',
  'number_of_stations': '3',
  'region': 'Virgin Islands region',
  'location': {'type': 'Point', 'coordinates': [-64.3056, 19.6145]}}]

In [17]:
df_n2 = pd.json_normalize(response, 
                           record_path=["location", "coordinates"],
                           meta=["depth", "earthquake_id", "magnitude", "number_of_stations",
                                 ["location", "type"], "source"]
                          )

df_n2.head()


,0,depth,earthquake_id,magnitude,number_of_stations,location.type,source
0,-64.3056,25,12258002,3.1,3,Point,pr
1,19.6145,25,12258002,3.1,3,Point,pr
2,-64.4080,43,12252002,3.1,6,Point,pr
3,19.6117,43,12252002,3.1,6,Point,pr
4,-63.8544,47,12256005,3.1,3,Point,pr


In [19]:
df_n2.rename(columns={0 : "coordinates"}, inplace=True)

df_n2.head()

,coordinates,depth,earthquake_id,magnitude,number_of_stations,location.type,source
0,-64.3056,25,12258002,3.1,3,Point,pr
1,19.6145,25,12258002,3.1,3,Point,pr
2,-64.4080,43,12252002,3.1,6,Point,pr
3,19.6117,43,12252002,3.1,6,Point,pr
4,-63.8544,47,12256005,3.1,3,Point,pr


In [24]:
coord = df_n2.iloc[:,0]
print(len(coord))
coord.head()

76


0   -64.3056
1    19.6145
2   -64.4080
3    19.6117
4   -63.8544
Name: coordinates, dtype: float64

In [38]:
coord_lng = coord[0:len(coord):2]
coord_lat = coord[1:len(coord):2]

In [32]:
df_n2v2 = df_n2.drop("coordinates", axis = 1)
df_n2v2.head()

,depth,earthquake_id,magnitude,number_of_stations,location.type,source
0,25,12258002,3.1,3,Point,pr
1,25,12258002,3.1,3,Point,pr
2,43,12252002,3.1,6,Point,pr
3,43,12252002,3.1,6,Point,pr
4,47,12256005,3.1,3,Point,pr


In [35]:
df_n2v2 = df_n2v2.drop_duplicates().copy()
df_n2v2.head()

,depth,earthquake_id,magnitude,number_of_stations,location.type,source
0,25,12258002,3.1,3,Point,pr
2,43,12252002,3.1,6,Point,pr
4,47,12256005,3.1,3,Point,pr
6,43,12257002,3.1,4,Point,pr
8,60,12252005,3.3,7,Point,pr


In [49]:
df_n2v2.loc[:, "longitude"] = coord_lng

In [41]:
coord_lat = coord_lat.to_list()

In [42]:
df_n2v2.insert(6, 'latitude', coord_lat, True)

In [43]:
print(len(df_n2v2))
df_n2v2.head()

38


,depth,earthquake_id,magnitude,number_of_stations,location.type,source,latitude,longitude,latitude
0,25,12258002,3.1,3,Point,pr,19.6145,-64.3056,NaN
2,43,12252002,3.1,6,Point,pr,19.6117,-64.4080,NaN
4,47,12256005,3.1,3,Point,pr,19.3323,-63.8544,NaN
6,43,12257002,3.1,4,Point,pr,19.1303,-64.5680,NaN
8,60,12252005,3.3,7,Point,pr,19.6184,-64.1700,NaN


#### json_normalize (record_path)

In [45]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]

In [47]:
pd.json_normalize(data)

,state,shortname,counties,info.governor
0,Florida,FL,"[{'name': 'Dade', 'population': 12345}, {'name...",Rick Scott
1,Ohio,OH,"[{'name': 'Summit', 'population': 1234}, {'nam...",John Kasich


In [48]:
pd.json_normalize(data, record_path=['counties'],
                 meta = ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


## Using nasted dataframe

In [51]:
df_n1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   source                38 non-null     object 
 1   earthquake_id         38 non-null     object 
 2   version               38 non-null     object 
 3   magnitude             38 non-null     object 
 4   depth                 38 non-null     object 
 5   number_of_stations    38 non-null     object 
 6   region                38 non-null     object 
 7   location.type         38 non-null     object 
 8   location.coordinates  38 non-null     object 
 9   longitude             38 non-null     float64
 10  latitude              38 non-null     float64
dtypes: float64(2), object(9)
memory usage: 3.4+ KB


In [57]:
import folium

df_n1['magnitude'] = df_n1['magnitude'].astype(float)
magnitude_mean = df_n1['magnitude'].mean()

map_center = [df_n1['latitude'].mean(), df_n1['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=5)

def color_marker(magnitude):
    if magnitude < magnitude_mean:
        return 'green'
    else:
        return 'red'

for index, row in df_n1.iterrows():
    magnitude = row['magnitude']
    color = color_marker(magnitude)
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Magnitude: {magnitude}",
        icon=folium.Icon(color=color)
    ).add_to(mymap)

mymap
